## Purpose:
The goal of this assignment is to develop a model that uses apriori analysis to identify patterns of words in the text. Along with this, is the comparison of handwriting dataset analysis using a single-layer neural network vs a multi-layer neural network. 

### Use Apriori analysis to idenfity patterns in the text (Alice in Wonderland by Lewis Carroll)

In [1]:
import nltk
import numpy as np
import pandas as pd
import re
import string
from itertools import combinations 
import csv
from nltk.corpus import gutenberg, stopwords

In [2]:
carroll_words = nltk.corpus.gutenberg.words('carroll-alice.txt')
print(f'Number of words: {len(carroll_words)}')

carroll_sentences = gutenberg.sents('carroll-alice.txt')
print(f'Number of sentences: {len(carroll_sentences)}')

Number of words: 34110
Number of sentences: 1703


### Clean stop words and symbols from sentences

In [3]:
# Formerly Items_names
Items_names = {}  # Lookup item ID to name

# Formerly Items_ids
Items_ids = {} # Lookup word name to ID

Transactions_list = []  # a list of transactions
item_id = 0

for terms in carroll_sentences:
    
    # Remove stop words
    terms = [w for w in terms if w not in stopwords.words('english') ]
    
    # Remove punctuations
    terms = [w for w in terms if w not in string.punctuation] 
    
    # Remove numbers
    terms = [w for w in terms if re.search(r'^[a-zA-Z]{2}', w) is not None]
    
    transaction = []
    for item in terms:
        if item not in Items_ids:
            Items_ids[item] = item_id
            Items_names[item_id] = item
            item_id += 1
        transaction += [Items_ids[item]]
    Transactions_list += [transaction]

In [4]:
M, N = len(Items_ids), len(Transactions_list)

Items = np.arange(0,M)

# Information, sanity
print(f'M={M} items, N={N} transactions')

M=2793 items, N=1703 transactions


In [5]:
# Sanity check
print([Items_names[_] for _ in Items[0:15]])
print(Transactions_list[:7])

['Alice', 'Adventures', 'Wonderland', 'Lewis', 'Carroll', 'CHAPTER', 'Down', 'Rabbit', 'Hole', 'beginning', 'get', 'tired', 'sitting', 'sister', 'bank']
[[0, 1, 2, 3, 4], [5], [6, 7, 8], [0, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 13, 19, 20, 21, 22, 18, 23, 0, 24, 20, 25], [26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 7, 50, 51, 52, 53], [54, 15, 55, 56, 0, 57, 55, 58, 59, 60, 7, 61, 62, 63], [62, 63]]


In [6]:
# Sanity check
print(Items_ids)

{'Alice': 0, 'Adventures': 1, 'Wonderland': 2, 'Lewis': 3, 'Carroll': 4, 'CHAPTER': 5, 'Down': 6, 'Rabbit': 7, 'Hole': 8, 'beginning': 9, 'get': 10, 'tired': 11, 'sitting': 12, 'sister': 13, 'bank': 14, 'nothing': 15, 'twice': 16, 'peeped': 17, 'book': 18, 'reading': 19, 'pictures': 20, 'conversations': 21, 'use': 22, 'thought': 23, 'without': 24, 'conversation': 25, 'So': 26, 'considering': 27, 'mind': 28, 'well': 29, 'could': 30, 'hot': 31, 'day': 32, 'made': 33, 'feel': 34, 'sleepy': 35, 'stupid': 36, 'whether': 37, 'pleasure': 38, 'making': 39, 'daisy': 40, 'chain': 41, 'would': 42, 'worth': 43, 'trouble': 44, 'getting': 45, 'picking': 46, 'daisies': 47, 'suddenly': 48, 'White': 49, 'pink': 50, 'eyes': 51, 'ran': 52, 'close': 53, 'There': 54, 'VERY': 55, 'remarkable': 56, 'think': 57, 'much': 58, 'way': 59, 'hear': 60, 'say': 61, 'Oh': 62, 'dear': 63, 'shall': 64, 'late': 65, 'afterwards': 66, 'occurred': 67, 'ought': 68, 'wondered': 69, 'time': 70, 'seemed': 71, 'quite': 72, 'natu

In [7]:
# Sanity check
print(Transactions_list)

[[0, 1, 2, 3, 4], [5], [6, 7, 8], [0, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 13, 19, 20, 21, 22, 18, 23, 0, 24, 20, 25], [26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 7, 50, 51, 52, 53], [54, 15, 55, 56, 0, 57, 55, 58, 59, 60, 7, 61, 62, 63], [62, 63], [64, 65], [23, 66, 67, 68, 69, 70, 71, 72, 73, 7, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 0, 84, 85, 86, 87, 28, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 52, 87, 98, 99, 70, 100, 101, 102, 90, 103, 104], [105, 106, 107, 108, 0, 88, 27, 109, 10], [110, 90, 103, 108, 111, 112, 113, 59, 114, 48, 48, 0, 107, 57, 115, 116, 117, 118, 29], [119, 29, 118, 120, 121, 122, 70, 108, 123, 124, 125, 126, 127], [128, 129, 123, 130, 131, 132, 100, 133, 82, 134, 29, 135, 136, 137, 18, 138, 139, 140, 20, 141, 142, 143], [144, 145, 146, 147, 138, 148, 149, 150, 151, 152, 153, 154, 112, 155, 146, 156, 157, 158, 159, 160, 147, 137, 120, 161], [162], [23, 0, 163, 64, 57, 15, 164, 165], [166, 167, 57, 168], [16

In [8]:
# Convert to numpy arrays
Transactions = np.full((N,M), False, dtype=bool)

In [9]:
# Show all columns of array
np.set_printoptions(threshold=np.inf)

In [10]:
# Convert to numpy arrays
Transactions = np.full((N,M), False, dtype=bool)

for i, t in enumerate(Transactions_list):
    for item in t:
        Transactions[i][item] = True

# Sanity, print row index 10, 11
print(f'{Transactions[10:12].astype(int)}')

[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1
  1 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [11]:
# Transfer files to .csv to be viewed on Weka FPGrowth for analysis
Filename = 'input_for_weka.csv'

with open(Filename, 'w') as fout:
    writer = csv.writer(fout, delimiter=',', quoting=csv.QUOTE_ALL, quotechar="'", lineterminator='\n')
    writer.writerow([Items_names[i] for i in range(M)])
    for i in range(N):
        writer.writerow(list(map(lambda x: '' if x == False else 'True',  Transactions[i])))

### Analysis

I ran the Apriori analysis using Weka Explorer’s FPGrowth associator, and generated analyses using the following parameters:

| Delta | Lower Bound Min Support | Max Number Of Items | Metric Type | Min Metric | Rules to Find | Rules Found |
| --- | --- | --- | --- | --- | --- | --- |
| 0.05 | 0.01 | -1 | Confidence | 0.9 | 10 | 7 |
| 0.05 | 0.005 | -1 | Confidence | 0.9 | 10 | 13 |
| 0.05 | 0.004 | -1 | Confidence | 0.9 | 10 | 20 |
| 0.05 | 0.003 | -1 | Confidence | 0.9 | 10 | 38 |
| 0.05 | 0.0025 | -1 | Confidence | 0.9 | 10 | 74 |
| 0.05 | 0.001 | -1 | Confidence | 0.9 | 10 | 531933 |

From my generated table, I wanted to find the “optimal” number of rules found when running an analysis based on confidence intervals. It appears that there is an “optimal” number of rules to be found when altering only the lowerBoundMinSupport to be within the range of 0.01 and 0.001. 

With lowerBoundMinSupport = 0.01, there were seven rules found. Of those seven, three of the pairs (six total items) were part of the same item set. Example pairs are below: 

### lowerBoundMinSupport = 0.01
| Antecedent | Consequent | Confidence |
| --- | --- | --- |
| Mock | Turtle | 1 |
| White | Rabbit | 1 |
| Hare | March | 1 |
| said | Turtle | 1 |
| said | Mock | 1 |
| March | Hare | 0.97 |
| Turtle | Mock | 0.97 |

It appears that the names of characters such as “Mock Turtle” and “March Hare” appeared most frequently with each other, which makes sense as the story revolves around the main characters. Similarly, speaking roles for these main characters would occur the most frequently, resulting in their names being associated with “said”. 

Taking a step forward by setting lowerBoundMinSupport = 0.05 results in 13 rules found. Similar to the previous settings, there were pairs of antecedent-consequent compliments. The new additions added were:

### lowerBoundMinSupport = 0.005
| Antecedent | Consequent | Confidence |
| --- | --- | --- |
| join | dance | 1 |
| said | White | 1 |
| said | Hare | 1 |
| said, March | Hare | 0.94 |
| Of | course | 0.91 |
| glass | little | 0.9 |

More statements associated with “said” were added. Then, it appears there are multiple instances of the affirmation “Of course”, the act of “join dance”, and descriptions of the “little glass”. These sets appears 9-10 times, compared to the first set which appears 30 times each.

When lowerBoundMinSupport = 0.003, rules begin to appear that have approximately nine or less occurrences in the set of associations. This results in more items set Antecdents of size two or greater. Furthermore, associations with "Alice" begin to appear more. Sets associated with “Alice” are words such as “thought, get”, “thought, might”, “Turtle”, “White”. At around lowerBoundMinSupport = 0.0025, there begins to appear items set Antecdents of size three or greater; with these sets occurring at a frequency of five or less. 

### Compare prediction results from single-layer neural network vs. multi-layer neural network

In [12]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import ranf
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

### Load MNIST files

In [13]:
import os
import struct
import numpy as np
import gzip
 
def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path, 
                               '%s-labels-idx1-ubyte.gz' % kind)
    images_path = os.path.join(path, 
                               '%s-images-idx3-ubyte.gz' % kind)
        
    with gzip.open(labels_path, 'rb') as lbpath:
        lbpath.read(8)
        buffer = lbpath.read()
        labels = np.frombuffer(buffer, dtype=np.uint8)

    with gzip.open(images_path, 'rb') as imgpath:
        imgpath.read(16)
        buffer = imgpath.read()
        images = np.frombuffer(buffer, 
                               dtype=np.uint8).reshape(
            len(labels), 784)
 
    return images, labels


X_train_mnist, y_train_mnist = load_mnist('Datasets/mnist/', kind='train')
print(f'Rows= {X_train_mnist.shape[0]}, columns= {X_train_mnist.shape[1]}')

X_test_mnist, y_test_mnist = load_mnist('Datasets/mnist', kind='t10k')
print(f'Rows= {X_test_mnist.shape[0]}, columns= {X_test_mnist.shape[1]}')

Rows= 60000, columns= 784
Rows= 10000, columns= 784


In [14]:
class NeuralNetMLP(object):

    def __init__(self, n_hidden=30, epochs=100, eta=0.001, minibatch_size=1, seed=None):
        self.random = np.random.RandomState(seed)  # used to randomize weights
        self.n_hidden = n_hidden  # size of the hidden layer
        self.epochs = epochs  # number of iterations
        self.eta = eta  # learning rate
        self.minibatch_size = minibatch_size  # size of training batch - 1 would not work
        self.w_out, self.w_h = None, None
    
    @staticmethod
    def onehot(_y, _n_classes):  # one hot encode the input class y
        onehot = np.zeros((_n_classes, _y.shape[0]))
        for idx, val in enumerate(_y.astype(int)):
            onehot[val, idx] = 1.0
        return onehot.T
    
    @staticmethod
    def sigmoid(_z):  # Eq 1
        return 1.0 / (1.0 + np.exp(-np.clip(_z, -250, 250)))

    def _forward(self, _X):  # Eq 2
        
        # step 1: net input of hidden layer
        z_h = np.dot(_X, self.w_h)
        
        # step 2: activation of hidden layer
        a_h = self.sigmoid(z_h)
        
        # step 3: net input of output layer
        z_out = np.dot(a_h, self.w_out)
        
        # step 4: activation output layer
        a_out = self.sigmoid(z_out)
        return z_h, a_h, z_out, a_out

    @staticmethod
    def compute_cost(y_enc, output):  # Eq 4
        term1 = -y_enc * (np.log(output))
        term2 = (1.0-y_enc) * np.log(1.0-output)
        cost = np.sum(term1 - term2)
        return cost

    def predict(self, _X):
        z_h, a_h, z_out, a_out = self._forward(_X)
        ypred = np.argmax(z_out, axis=1)
        return ypred

    def fit(self, _X_train, _y_train, _X_valid, _y_valid):
        import sys
        n_output = np.unique(_y_train).shape[0]  # number of class labels
        n_features = _X_train.shape[1]
        self.w_out = self.random.normal(loc=0.0, scale=0.1, size=(self.n_hidden, n_output))
        self.w_h = self.random.normal(loc=0.0, scale=0.1, size=(n_features, self.n_hidden))
        y_train_enc = self.onehot(_y_train, n_output)  # one-hot encode original y
        for ei in range(self.epochs):  # Ideally must shuffle at every epoch
            indices = np.arange(_X_train.shape[0])
            for start_idx in range(0, indices.shape[0] - self.minibatch_size + 1, self.minibatch_size):
                batch_idx = indices[start_idx:start_idx + self.minibatch_size]               
                z_h, a_h, z_out, a_out = self._forward(_X_train[batch_idx])  # neural network model                
                sigmoid_derivative_h = a_h * (1.0-a_h)  # Eq 3
                delta_out = a_out - y_train_enc[batch_idx]  # Eq 5
                delta_h = (np.dot(delta_out, self.w_out.T) * sigmoid_derivative_h)  # Eq 6
                grad_w_out = np.dot(a_h.T, delta_out)  # Eq 7
                grad_w_h = np.dot(_X_train[batch_idx].T, delta_h)  # Eq 8
                self.w_out -= self.eta*grad_w_out  # Eq 9
                self.w_h -= self.eta*grad_w_h  # Eq 9

            # Evaluation after each epoch during training
            z_h, a_h, z_out, a_out = self._forward(_X_train)
            cost = self.compute_cost(y_enc=y_train_enc, output=a_out)
            y_train_pred = self.predict(_X_train)  # monitoring training progress through reclassification
            y_valid_pred = self.predict(_X_valid)  # monitoring training progress through validation
            train_acc = ((np.sum(_y_train == y_train_pred)).astype(float) / _X_train.shape[0])
            valid_acc = ((np.sum(_y_valid == y_valid_pred)).astype(float) / _X_valid.shape[0])
            sys.stderr.write('\r%d/%d | Cost: %.2f ' '| Train/Valid Acc.: %.2f%%/%.2f%% '%
                (ei+1, self.epochs, cost, train_acc*100, valid_acc*100))
            sys.stderr.flush()
        return self

### 2-Layer Neural Network

In [16]:
class NeuralNetMLP_2Layers(object):

    def __init__(self, n_hidden=30, epochs=100, eta=0.001, minibatch_size=1, seed=None):
        self.random = np.random.RandomState(seed)  # used to randomize weights
        self.n_hidden = n_hidden  # size of the hidden layer
        self.epochs = epochs  # number of iterations
        self.eta = eta  # learning rate
        self.minibatch_size = minibatch_size  # size of training batch - 1 would not work
        self.w_out, self.w1_h, self.w2_h = None, None, None
    
    @staticmethod
    def onehot(_y, _n_classes):  # one hot encode the input class y
        onehot = np.zeros((_n_classes, _y.shape[0]))
        for idx, val in enumerate(_y.astype(int)):
            onehot[val, idx] = 1.0
        return onehot.T
    
    @staticmethod
    def sigmoid(_z):  # Eq 1
        return 1.0 / (1.0 + np.exp(-np.clip(_z, -250, 250)))

    def _forward(self, _X):  # Eq 2
        
        # hidden layer 1 output
        z1_h = np.dot(_X, self.w1_h)

        # activation of hidden layer 1
        a1_h = self.sigmoid(z1_h)

        # hidden layer 2 output
        z2_h = np.dot(a1_h, self.w2_h)

        # activation of hidden layer 2
        a2_h = self.sigmoid(z2_h)
        
        # net input of output layer
        z_out = np.dot(a2_h, self.w_out)
        a_out = self.sigmoid(z_out)
        
        return z1_h, a1_h, z2_h, a2_h, z_out, a_out

    @staticmethod
    def compute_cost(y_enc, output):  # Eq 4
        term1 = -y_enc * (np.log(output))
        term2 = (1.0-y_enc) * np.log(1.0-output)
        cost = np.sum(term1 - term2)
        return cost

    def predict(self, _X):
        z1_h, a1_h, z2_h, a2_h, z_out, a_out = self._forward(_X)
        ypred = np.argmax(z_out, axis=1)
        return ypred

    def fit(self, _X_train, _y_train, _X_valid, _y_valid):
        import sys
        n_output = np.unique(_y_train).shape[0]  # number of class labels
        n_features = _X_train.shape[1]
        self.w_out = self.random.normal(loc=0.0, scale=0.1, size=(self.n_hidden, n_output))
        self.w1_h = self.random.normal(loc=0.0, scale=0.1, size=(n_features, self.n_hidden))
        self.w2_h = self.random.normal(loc=0.0, scale=0.1, size=(self.n_hidden, self.n_hidden))
        y_train_enc = self.onehot(_y_train, n_output)  # one-hot encode original y
        for ei in range(self.epochs):  # Ideally must shuffle at every epoch
            indices = np.arange(_X_train.shape[0])
            for start_idx in range(0, indices.shape[0] - self.minibatch_size + 1, self.minibatch_size):
                batch_idx = indices[start_idx:start_idx + self.minibatch_size]               
                z1_h, a1_h, z2_h, a2_h, z_out, a_out = self._forward(_X_train[batch_idx])  # neural network model                
                
                sigmoid_derivative_h1 = a1_h * (1.0-a1_h)  # Eq 3
                sigmoid_derivative_h2 = a2_h * (1.0-a2_h)  # Eq 3
                                
                delta_out = a_out - y_train_enc[batch_idx]  # Eq 5
                
                delta_h2 = (np.dot(delta_out, self.w_out.T) * sigmoid_derivative_h2)  # Eq 6
                delta_h1 = (np.dot(delta_h2, self.w2_h.T) * sigmoid_derivative_h1)    # Eq 6
                                
                grad_w2_out = np.dot(a2_h.T, delta_out) # Eq 7
                grad_w1_out = np.dot(a1_h.T, delta_h2)  # Eq 7
                                  
                grad_w1_h = np.dot(_X_train[batch_idx].T, delta_h1)  # Eq 8
                grad_w2_h = np.dot(a1_h.T, delta_h2)                 # Eq 8
                                
                self.w_out -= self.eta*grad_w2_out  # Eq 9
                self.w1_h -= self.eta*grad_w1_h     # Eq 9 
                self.w2_h -= self.eta*grad_w2_h     # Eq 9
                
            # Evaluation after each epoch during training
            z1_h, a1_h, z2_h, a2_h, z_out, a_out = self._forward(_X_train)
            cost = self.compute_cost(y_enc=y_train_enc, output=a_out)
            y_train_pred = self.predict(_X_train)  # monitoring training progress through reclassification
            y_valid_pred = self.predict(_X_valid)  # monitoring training progress through validation
            train_acc = ((np.sum(_y_train == y_train_pred)).astype(float) / _X_train.shape[0])
            valid_acc = ((np.sum(_y_valid == y_valid_pred)).astype(float) / _X_valid.shape[0])
            sys.stderr.write('\r%d/%d | Cost: %.2f ' '| Train/Valid Acc.: %.2f%%/%.2f%% '%
                (ei+1, self.epochs, cost, train_acc*100, valid_acc*100))
            sys.stderr.flush()
        return self

### 1-Layer - 300 epochs

In [15]:
# Define and fit the neural network
nn = NeuralNetMLP(n_hidden=20, epochs=300, eta=0.0005, minibatch_size=100, seed=1)

nn.fit(X_train_mnist[:55000], y_train_mnist[:55000], X_train_mnist[55000:], y_train_mnist[55000:]) ;

300/300 | Cost: 32420.83 | Train/Valid Acc.: 90.86%/92.62% 

### 2-Layer - 300 epochs

In [17]:
# Define and fit the neural network
nn = NeuralNetMLP_2Layers(n_hidden=20, epochs=300, eta=0.0005, minibatch_size=100, seed=1)

nn.fit(X_train_mnist[:55000], y_train_mnist[:55000], X_train_mnist[55000:], y_train_mnist[55000:]) ;

300/300 | Cost: 27624.69 | Train/Valid Acc.: 91.76%/93.18% 

Slightly improved performance with the 2-hidden-layer MLP, with an increase of approximately 1% improvement. 